In [62]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from collections import defaultdict
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch import device

from lime.lime_text import LimeTextExplainer
from transformers import RobertaTokenizer
from src.roberta.roberta_tokenizer import TokenizedDataset
from src.roberta.roberta_model import RobertaClassification
from src.preprocessor import balancing_multiple_classes
from src.roberta.roberta_functions import *

In [24]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',None)

In [44]:
device: device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [38]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [37]:
emotion_list = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

In [25]:
data_dir =  os.path.join(os.path.sep.join(os.getcwd().split(os.path.sep)[:-1]), "data")
models_dir =  os.path.join(os.path.sep.join(os.getcwd().split(os.path.sep)[:-1]), "models")

In [39]:
df_test = pd.read_csv(os.path.join(data_dir,"test_set.csv"))
emotion_dummies_test = pd.get_dummies(df_test['emotion'])
emotion_dummies_test = emotion_dummies_test.astype(int)
df_test_main = pd.concat([df_test['sentence'],  emotion_dummies_test], axis=1)
test_dataset = TokenizedDataset(df_test_main, tokenizer, 256, emotion_list)

In [41]:
test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=32,
                                               shuffle=False,
                                               num_workers=0
                                               )

In [42]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [45]:
model = RobertaClassification(num_labels=6)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClassification(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
               

In [46]:
test_dataset = TokenizedDataset(df_test_main, tokenizer, 256, emotion_list)
test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=32,
                                               shuffle=False,
                                               num_workers=0
                                               )

In [ ]:
def get_predictions(model, data_loader):
    """
    Obtain predictions from a model based on the input from a data loader.

    This function processes batches of sentences from the data loader, performs
    predictions using the provided model, and gathers the outputs including sentences,
    predictions, prediction probabilities, and emotion values.

    Args:
        model (torch.nn.Module): The model to use for predictions, expected to be in evaluation mode.
        data_loader (DataLoader): A PyTorch DataLoader containing the dataset for prediction.

    Returns:
        tuple: A tuple containing four lists:
            - sentences (list[str]): The original sentences from the data loader.
            - predictions (list[int]): The predicted class indices for each sentence.
            - prediction_probs (list[list[float]]): The probabilities for each class for each sentence.
            - emotion_values (list[int]): The emotion values extracted from the dataset.
    """
    model.eval()

    sentences = []
    predictions = []
    prediction_probs = []
    emotion_values = []

    with torch.no_grad():
        for data in data_loader:
            sentence = data["sentence"]
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            emotions = data["emotion"].to(device, dtype=torch.float)

            outputs = model(ids, mask)
            _, preds = torch.max(outputs, dim=1)
            _, emotion = torch.max(emotions, dim=1)

            sentences.extend(sentence)
            predictions.extend(preds.tolist())  # Convert tensor to list
            prediction_probs.extend(outputs.tolist())  # Convert tensor to list
            emotion_values.extend(emotions.argmax(dim=1).tolist())  # Convert tensor to list

    return sentences, predictions, prediction_probs, emotion_values

In [47]:
sentences, predictions, prediction_probs, emotion_values = get_predictions(model, test_data_loader, device)

KeyboardInterrupt: 

In [50]:
submission = pd.read_csv(os.path.join(data_dir,"roberta_test2.csv"))
submission.head()

,id,emotion
0,0,happiness
1,1,surprise
2,2,disgust
3,3,disgust
4,4,surprise


In [57]:
joined_df = df_test.join(submission[['emotion']], rsuffix='_df_test')

diff_df = joined_df[joined_df['emotion'] != joined_df['emotion_df_test']]

In [63]:
diff_df

,id,sentence,emotion,emotion_df_test
4,4,"I can�t believe it, they lost the game in the last minute.",sadness,surprise
12,12,"When I do not get something right the first time, I get frustrated.",sadness,anger
15,15,Paul started to resent Lola after all the broken promises.,sadness,anger
20,20,The mere thought of ghosts wandering in the attic gives me goosebumps.,disgust,fear
50,50,I have a strong aversion against vegetables.,sadness,disgust
...,...,...,...,...
1413,1413,Hearing the unexpected news about the job promotion shook me with excitement.,surprise,happiness
1419,1419,Some drivers on the road drive like they got their driver's license in kinder garden.,anger,disgust
1426,1426,The Doctor's results are worrying.,sadness,fear
1432,1432,Her ability to recall obscure facts never ceases to amaze her friends.,surprise,disgust


In [64]:
explainer = LimeTextExplainer(class_names=emotion_list)

In [67]:
sen = diff_df.iloc[0]['sentence']
sen

'I can�t believe it, they lost the game in the last minute.'

In [ ]:
#exp = explainer.explain_instance(sen, predict_proba, num_features=6)
#exp.show_in_notebook(text=True)
